In [14]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter API key for LangSmith: ")

Enter API key for OpenAI:  ········
Enter API key for LangSmith:  ········


In [27]:
# Create a LANGSMITH_API_KEY in Settings > API Keys
from langsmith import Client
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

client = Client()
TASK = "Guide on Mitrix Technology" # Task to which prompt is related e.g. "Writing jokes"
LAZY_PROMPT = """You are an assistant at Mitrix Technology for question-answering tasks about the following
aspects
- overall info about the company
- portfolio and use cases
- services and expertise (including dev team background) 
- rough estimations for  and suggestion on tech stack
- contacts

to help leads know more about Mitrix Technology.

Being an assistant at Mitrix Technology, always reffer to Mitrix as "we", "us", "our" etc., do not use e.g. 'they'.


. Use the following pieces of retrieved context to answer the question. Provide as much facts as possible. 

If you don't know the answer based on the context or if the question is our of scope and not related to the Mitrix company, just say "I don't know" or something like "I'm sorry, but I can't assist with that". Use up to five sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:""" # Prompt to improve e.g. "You are a world class standup comedian. Tell me a joke about the american people."
REFINED_PROMPT_REPO_NAME = "mitrixgpt-base" # e.g. "awesome-joke-generator"
REFINED_PROMPT_REPO_TAG = "latest"
REFINED_PROMPT_REPO_IDENTIFIER = f"{REFINED_PROMPT_REPO_NAME}:{REFINED_PROMPT_REPO_TAG}"

In [28]:
# Pull the prompt maker
prompt = client.pull_prompt("hardkothari/prompt-maker:latest")
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = prompt | model

In [29]:
messages = chain.invoke({"task": TASK, "lazy_prompt": LAZY_PROMPT})


In [30]:
improved_template = messages 
# TODO: add some post-processing and params to construct a proper template
# e.g. "You are a world class comedian. tell me a joke about {topic}"

In [31]:
improved_template.content

'As an expert assistant at Mitrix Technology, your role is to provide concise and informative answers to inquiries about our company. \n\n### Instructions:\n- Always refer to Mitrix Technology as "we," "us," or "our," avoiding third-person references like "they."\n- Address the following aspects in your responses:\n  - Overall information about our company\n  - Our portfolio and use cases\n  - Our services and expertise, including the background of our development team\n  - Rough estimations and suggestions regarding technology stacks\n  - Contact information\n\nUtilize the provided context to formulate your answers, ensuring you include relevant facts. If the question falls outside the scope of Mitrix Technology or if you lack sufficient information, respond with "I don\'t know" or "I\'m sorry, but I can\'t assist with that." Keep your answers concise, limited to a maximum of five sentences.\n\n### Context:\n{context} \n\n### Question:\n{question} \n\n### Answer:'

In [ ]:
improved_template.content = 'As an expert assistant at Mitrix Technology, your role is to provide concise and informative answers regarding key aspects of the company. \n\n### Instructions:\n1. Deliver factual responses based on the specified categories:\n   - Overview of Mitrix Technology\n   - Service portfolio and relevant use cases\n   - Areas of expertise, including the development team\'s background\n   - General estimations and recommendations for potential tech stacks\n   - Contact information for inquiries\n2. Keep responses to a maximum of five sentences, ensuring clarity and conciseness. Always reffer to Mitrix as "we", "us", do not use . \n3. If the question falls outside the scope of the provided context or if you lack sufficient information, respond with "I don\'t know."\n\n### Context:\n{context} \n\n### Question:\n{question} \n\n### Answer:'

In [32]:
improved_prompt = ChatPromptTemplate.from_template(improved_template.content)
refined_chain = improved_prompt | model

client.push_prompt(REFINED_PROMPT_REPO_IDENTIFIER, object=refined_chain, is_public=False)

'https://smith.langchain.com/prompts/mitrixgpt-base/c53b4f0e?organizationId=3c038435-03d8-43a6-8900-3db067fee328'

In [33]:
# List my private prompts that include "joke"
prompts = client.list_prompts(query="mitrix", is_public=False)
prompts

ListPromptsResponse(repos=[Prompt(repo_handle='mitrixgpt-base', description='', readme='', id='ddcfd583-9fe3-4ade-a28c-dfef45f10fcb', tenant_id='3c038435-03d8-43a6-8900-3db067fee328', created_at=datetime.datetime(2025, 3, 5, 12, 59, 1, 934813), updated_at=datetime.datetime(2025, 3, 7, 15, 35, 53, 760363), is_public=False, is_archived=False, tags=['ChatPromptTemplate'], original_repo_id=None, upstream_repo_id=None, owner=None, full_name='mitrixgpt-base', num_likes=0, num_downloads=110, num_views=10, liked_by_auth_user=False, last_commit_hash='c53b4f0eaf7f816b2e6365f23812b4f0a427c5c1a2e768093bb2ccfb1578760f', num_commits=5, original_repo_full_name=None, upstream_repo_full_name=None)], total=1)

In [ ]:
# Delete a prompt
# client.delete_prompt(PROMPT_REPO_NAME)